In [1]:
#import libraries
from sys import modules
from os import listdir
import pandas as pd
import numpy as np
if not "seaborn" in modules:
    %pip install seaborn
import seaborn as sns
if not "matplotlib" in modules:
    %pip install matplotlib
import matplotlib as plt
if not "xgboost" in modules:
    %pip install xgboost
from xgboost import XGBClassifier
if not "sklearn" in modules:
    %pip install sklearn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
if not "kaggle" in modules:
    %pip install kaggle
import kaggle

# random state to be used for the whole program
random_state = 3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [36]:
# set up kaggle API access
# make sure to upload your API key, which should be in a file called "kaggle.json"
if "kaggle.json" in listdir():
    if not ".kaggle" in listdir():
        %mkdir ~/.kaggle/
    if not ".kaggle" in listdir(".kaggle"):
        %mv kaggle.json ~/.kaggle/

# copy this into the terminal
# chmod 600 ~/.kaggle/kaggle.json

UsageError: Line magic function `%chmod` not found.


You can download the dataset [here](https://www.kaggle.com/code/arjunjoshua/predicting-fraud-in-financial-payment-services/input).

In [2]:
df = pd.read_csv("PS_20174392719_1491204439457_log.csv")

In [7]:
df.shape
# (6362620, 11)

(6362620, 11)

In [ ]:
# see if anything pops out with basic stats
df.describe()

In [9]:
# look at data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [ ]:
df.columns
# step
# Maps a unit of time in the real world. In this case 1 step is 1 hour of time.

# type
# CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER

# amount
# amount of the transaction in local currency

# nameOrig
# customer who started the transaction

# oldbalanceOrg
# initial balance before the transaction

# newbalanceOrg
# customer's balance after the transaction.

# nameDest
# recipient ID of the transaction.

# oldbalanceDest
# initial recipient balance before the transaction.

# newbalanceDest
# recipient's balance after the transaction.

# isFraud
# identifies a fraudulent transaction (1) and non fraudulent (0)

# isFlaggedFraud
# flags illegal attempts to transfer more than 200.000 in a single transaction.

In [3]:
# find out how much fraud occurs
n_frauds = df[df["isFraud"] == 1].shape[0]
print(f'{n_frauds} frauds out of {df.shape[0]}')
print(f"{n_frauds/df.shape[0]} %")
# 8213 frauds out of 6362620
# 0.001290820448180152 %

8213 frauds out of 6362620
0.001290820448180152 %


In [4]:
# find out the success rate for flagging transactions as fraud
mask1 = (df["isFlaggedFraud"] == 1) & (df["isFraud"] == 1)
correct_fraud_flags = df[mask1].shape[0]
mask2 = (df["isFlaggedFraud"] == 0) & (df["isFraud"] == 1)
incorrect_fraud_flags = df[mask2].shape[0]
print(f"                    frauds: {n_frauds}")
print(f"  frauds flagged correctly: {correct_fraud_flags}")
print(f"frauds flagged incorrectly: {incorrect_fraud_flags}")
fraud_flag_success_rate = 100 * correct_fraud_flags / n_frauds
print(f"   fraud flag success rate: {round(fraud_flag_success_rate, 3)}%")

                    frauds: 8213
  frauds flagged correctly: 16
frauds flagged incorrectly: 8197
   fraud flag success rate: 0.195%


In [ ]:
# see if any columns should be converted to numerical values
for column in df.columns:
    print(column, df.dtypes[column], df[column].unique()[:5])

In [ ]:
# see if anything pops out correlation-wise
corr = df.corr()
# corr
# ax = sns.heatmap(
#     corr, 
#     vmin=-1, vmax=1, center=0,
#     cmap=sns.diverging_palette(20, 220, n=200),
#     square=True
# )
# ax.set_xticklabels(
#     ax.get_xticklabels(),
#     rotation=45,
#     horizontalalignment='right'
# );

In [10]:
X = df[df.columns.drop(["isFraud", "isFlaggedFraud"])].select_dtypes(
    include=["int64", "float64"]
)
y = df["isFraud"]

In [12]:
X.sample(5)

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
4267372,307,133968.30,39786.00,0.0,1663576.18,1797544.48
2890300,228,11039.48,0.00,0.0,0.00,0.00
1694545,159,422920.50,170973.00,593893.5,0.00,0.00
3924708,285,3400051.36,20850.62,0.0,4303101.64,7703153.00
3409401,255,857.48,0.00,0.0,0.00,0.00


In [13]:
# use xg boost
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=.25,
    random_state=random_state
)

In [19]:
parameter_grid_rscv = {
    "n_estimators": [50, 100, 150],
    "learning_rate": [0.1, 0.05, 0.01],
    "booster": ["gbtree", "gblinear", "dart"],
    "n_jobs": [-1],
    "reg_alpha": [0.1, 0.05, 0.01],
    "reg_lambda": [0.1, 0.05, 0.01],
    "random_state": [random_state]
}
permutations = 1
for k in parameter_grid_rscv:
    permutations *= len(parameter_grid_rscv[k])
print(permutations)
    

243


In [26]:
xgb = XGBClassifier()

In [21]:
randomized_search_CV = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=parameter_grid_rscv,
    random_state=random_state,
    cv=3,
    n_iter=10,
    n_jobs=-1
)

In [22]:
# 151 minutes
rscv = randomized_search_CV.fit(
    X=X_train,
    y=y_train
)

In [23]:
print(rscv.best_params_)
# {'reg_lambda': 0.01, 'reg_alpha': 0.1, 'random_state': 3, 'n_jobs': -1, 'n_estimators': 150, 'learning_rate': 0.1, 'booster': 'dart'}

{'reg_lambda': 0.01, 'reg_alpha': 0.1, 'random_state': 3, 'n_jobs': -1, 'n_estimators': 150, 'learning_rate': 0.1, 'booster': 'dart'}


In [24]:
xgb_rscv = XGBClassifier(
    n_estimators=150,
    learning_rate=0.1,
    booster="dart",
    reg_alpha=0.1,
    reg_lambda=0.01,
    n_jobs=-1,
    random_state=random_state
)

In [36]:
parameter_grid_gscv = {
    "n_estimators": [125, 150, 175],
    "learning_rate": [0.1, 0.12, 0.08],
    "booster": ["gbtree", "gblinear", "dart"],
    "n_jobs": [-1],
    "reg_alpha": [0.1, 0.08, 0.12],
    "reg_lambda": [0.012, 0.008, 0.01],
    "random_state": [random_state]
}

In [37]:
grid_search_cv = GridSearchCV(
    estimator=xgb,
    param_grid=parameter_grid_gscv,
    cv=3,
    n_jobs=-1
)

In [ ]:
gscv = grid_search_cv.fit(
    X=X_train,
    y=y_train
)

In [ ]:
print(gscv.best_params_)